<a href="https://colab.research.google.com/github/NikoriakViktot/Data_Science_Course_SSWU/blob/main/WS1_Regression_Nikoriak.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Інформація про датасет

**Інформація про набір даних:**<br>
Набір даних містить 9358 усереднених за годину відповідей від масиву з 5 хімічних датчиків оксиду металу, вбудованих у хімічний мультисенсорний пристрій якості повітря. Апарат був розташований на полі в сильно забрудненому районі, на рівні дороги, в межах італійського міста. Дані були записані з березня 2004 р. по лютий 2005 р. (один рік), що представляє собою найдовші доступні у вільному доступі записи відповідей хімічних датчиків якості повітря, розміщених у полі. Ground Truth усереднені за годину концентрації CO, неметанових вуглеводнів, бензолу, загальних оксидів азоту (NOx) і діоксиду азоту (NO2) були надані суміжним еталонним сертифікованим аналізатором. <br>Відсутні значення позначено значенням -200.<br>
Інформація про атрибути:<br>

0. Date - Дата (ДД/ММ/РРРР)
1. Time - Час (ГГ.ХХ.СС)
2. CO(GT) Справжня середньогодинна концентрація СО в мг/м^3 (еталонний аналізатор)
3. PT08.S1 (оксид олова) усереднена за годину відповідь датчика (номінально цільовий CO)
4. NMHC(GT) - Справжня середньогодинна загальна концентрація неметанових вуглеводнів у мкг/м^3 (еталонний аналізатор)
5. C6H6(GT) - Справжня середньогодинна концентрація бензолу в мкг/м^3 (еталонний аналізатор)
6. PT08.S2 (titania) усереднена за годину відповідь датчика (номінально націлений NMHC)
7. NOx(GT) - Справжня середньогодинна концентрація NOx в ppb (еталонний аналізатор)
8. PT08.S3 (оксид вольфраму) погодинна усереднена відповідь датчика (номінально цільовий NOx)
9. NO2(GT) - Справжня середньогодинна концентрація NO2 в мкг/м^3 (еталонний аналізатор)
10. PT08.S4 (оксид вольфраму) погодинна усереднена відповідь датчика (номінально націлений NO2)
11. PT08.S5 (оксид індію) усереднена за годину відповідь датчика (номінально цільовий O3)
12. T - Температура в °C
13. RH - Відносна вологість (%)
14. AH - Абсолютна вологість



# Ініціалізація середовища

In [ ]:
# !pip install regressors

In [ ]:
# init environment
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen

import numpy as np
import pandas as pd
import scipy.stats as st
import math
import matplotlib.text as plttxt

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
plt.style.use('bmh')

from dateutil import parser

from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from sklearn.linear_model import BayesianRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import r2_score

from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import RepeatedKFold, cross_validate
from sklearn.compose import TransformedTargetRegressor, make_column_transformer
from sklearn.metrics import PredictionErrorDisplay, median_absolute_error, mean_absolute_error
import scipy as sp
from sklearn.linear_model import RidgeCV, LassoCV
# from regressors.stats import coef_pval
%matplotlib inline

# Зчитування і форматування даних

In [ ]:

file_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip'
resp = urlopen(file_url)
zipfile = ZipFile(BytesIO(resp.read()))
data = zipfile.open('AirQualityUCI.csv')
df = pd.read_csv(data, sep=';')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df.head(3)

In [ ]:
# check empty rows and drop them
print('Initial shape: ' + str(df.shape))
print(df[df['Date'].isnull()])
df.dropna(how='all', inplace=True)
print('Result shape: ' + str(df.shape))

In [ ]:
# check if the last 2 columns has values
[df['Unnamed: 16'].unique(),  df['Unnamed: 16'].unique()]

In [ ]:
# drop last 2 columns
df.drop(['Unnamed: 15', 'Unnamed: 16'], axis=1, inplace=True)
df.head()

In [ ]:
# check data
df.dtypes

In [ ]:
df.info()

Як бачимо частина параметрів, що мала б бути числовими, такими не є. Виправимо це.

In [ ]:
df['DateTime'] = (df['Date'] + ' ' + df['Time'].str.replace('.', ':')).map(parser.parse)
df['Date'] = df['Date'].map(parser.parse)
df['Hour'] = df['Time'].map(lambda x: int(x[0:2]))
df['C6H6(GT)'] = df['C6H6(GT)'].map(lambda x: float(x.replace(',','.')))
df['CO(GT)'] = df['CO(GT)'].map(lambda x: float(x.replace(',','.')))
df['T'] = df['T'].map(lambda x: float(x.replace(',','.')))
df['RH'] = df['RH'].map(lambda x: float(x.replace(',','.')))
df['AH'] = df['AH'].map(lambda x: float(x.replace(',','.')))

In [ ]:
df.dtypes

In [ ]:
df.dtypes

Перевіримо основні статистики данних

In [ ]:
df.describe(include='all')

За умовою, значення -200 - це пропущенні значення. Замінимо їх на nan

In [ ]:
df.replace(-200, np.nan, inplace=True)

Перевіримо описові статистики

In [ ]:
df.describe()

Перевіримо чи є порожні рядки (крім поля Hour)

In [ ]:
df.dropna(thresh=5, inplace=True)
df.describe()

In [ ]:
df.columns

In [ ]:

plt.figure(figsize=(14, 6))
sns.heatmap(df.isnull(), cbar=False, cmap='viridis', yticklabels=False)
plt.title("Missing Values Heatmap")
plt.xlabel("Features (Columns)")
plt.ylabel("Rows")
plt.tight_layout()
plt.show()


На графіку показано наявність пропущених значень (NaN) у кожному стовпці  датасету.
Жовті лінії позначають місця, де значення відсутні, а фіолетові — де значення є.
 Основні спостереження:
Колонка NMHC(GT) має найбільше пропусків (~90%), що видно як суцільна жовта смуга → її слід видалити, оскільки вона майже не містить корисної інформації.

Інші колонки, зокрема CO(GT), NOx(GT), NO2(GT) — мають помірну кількість пропусків (~17–18%), але вони містять важливу інформацію і можуть бути заповнені (імпутовані) медіаною.

Деякі сенсорні дані (PT08.*) і метеопараметри (T, RH, AH) мають незначні пропуски (~3.5%) → не критично, легко обробляється.

Графік допоміг визначити колонки, які варто видалити або обробити, щоб підвищити якість даних перед побудовою моделі.
Це важливий етап у процесі підготовки даних (data preprocessing), оскільки моделі машинного навчання не працюють із NaN без попередньої обробки.

In [ ]:
df.isnull().sum().sort_values(ascending=False)

Перевіримо розподіли

In [ ]:
dfm = np.log(df.iloc[:, 2:15]).melt(var_name='columns')
g = sns.displot(data=dfm, x='value', col='columns', col_wrap=3, common_norm=False, kde=True, stat='density')

Ці графіки показали, що деякі ознаки мають скошений розподіл, а отже:

Необхідно логарифмувати або нормалізувати деякі змінні (особливо цільову: C6H6(GT))

Викиди є, але масштаб не критичний

Сенсори (PT08.S*) добре підходять до моделі, бо мають стабільні розподіли

Графіки допомагають приймати рішення про попередню обробку (preprocessing)

In [ ]:
df["CO(GT)"] = df["CO(GT)"].fillna(df["CO(GT)"].median())

In [ ]:
X = df.iloc[:, 2:15]
print(X.info())
imputer = SimpleImputer(missing_values=np.nan, strategy='median')
imputer = imputer.fit(X)

X = imputer.transform(X)
df.iloc[:, 2:15] = X
df.describe()

заповнен відсутні значення в основних числових колонках за допомогою медіани, оскільки деякі змінні мають асиметричний розподіл (наприклад, CO(GT), NOx(GT)).
Медіана менш чутлива до викидів, ніж середнє, тому це дозволяє зберегти достовірність даних перед моделюванням.
Після заповнення всі колонки мають однакову кількість рядків (count = 9326), що означає, що набір готовий до аналізу та побудови моделей.

In [ ]:
dfm = np.log(df.iloc[:, 2:15]).melt(var_name='columns')
g = sns.displot(data=dfm, x='value', col='columns', col_wrap=3, common_norm=False, kde=True, stat='density')

Для покращення статистичних властивостей ознак було застосовано логарифмічне перетворення числових змінних.
Це дозволяє зменшити асиметрію розподілу, знизити вплив викидів та стабілізувати дисперсію, що в свою чергу сприяє покращенню якості побудови регресійних моделей і алгоритмів машинного навчання.

In [ ]:
df.dtypes

In [ ]:
df.head()

# EDA

Визначемо цільовий та вхідні параметри. Цільовий C6H6(GT).

In [ ]:
sns.pairplot(df)

In [ ]:
sns.pairplot(df[["CO(GT)", "C6H6(GT)", "T", "AH"]])


Для первинного аналізу було побудовано матрицю парних діаграм (pairplot) для основних змінних: CO(GT), C6H6(GT), температура (T) та відносна вологість (RH). Такий графік дозволяє візуально оцінити розподіли ознак та зв’язки між ними.

Основні спостереження:
Сильна позитивна кореляція між CO(GT) і C6H6(GT). Зі збільшенням концентрації оксиду вуглецю спостерігається зростання концентрації бензолу. Це може свідчити про спільне джерело забруднення, наприклад, транспорт.

Розподіли CO(GT) і C6H6(GT) скошені вправо — більшість значень зосереджена в нижньому діапазоні, з наявністю окремих високих значень (викидів).

Температура (T) і відносна вологість (RH) мають більш симетричні розподіли, що наближаються до нормальних.

Негативна залежність між температурою і вологістю: зі зростанням температури відносна вологість зменшується, що узгоджується з фізичними закономірностями атмосферних процесів.

Відсутність вираженої кореляції між CO(GT), C6H6(GT) та температурою або вологістю.

Висновок:
Отримані результати допомагають виділити змінні, які потенційно важливі для побудови моделей, а також звернути увагу на можливу мультиколінеарність, зокрема між CO та C6H6. Це слід враховувати при виборі ознак і типу моделі.

In [ ]:
# Створення кореляційної матриці для числових ознак
corr_matrix = df.iloc[:, 2:15].corr()

# Побудова теплової карти
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", square=True, linewidths=0.5)
plt.title("Кореляційна матриця числових змінних")
plt.show()


### Кореляційний аналіз змінних

Побудувана кореляційна матриця для числових змінних, виділено найбільш корельовані пари. Це дозволяє:

-  **Виявити сильні зв’язки між ознаками**;
-  **Зменшити кількість зайвих (схожих) змінних**;
-  **Підвищити якість моделі, уникнувши мультиколінеарності**.

---

####  Найсильніші корельовані пари:

| Змінна 1         | Змінна 2          | Кореляція |
|------------------|-------------------|------------|
| C6H6(GT)         | PT08.S2(NMHC)     | **0.98**   |
| PT08.S1(CO)      | PT08.S2(NMHC)     | **0.89**   |
| C6H6(GT)         | PT08.S5(O3)       | **0.87**   |
| CO(GT)           | C6H6(GT)          | **0.81**   |
| PT08.S3(NOx)     | PT08.S5(O3)       | **-0.80**  |

---

###  Висновок:

Цей аналіз допомагає відібрати **найінформативніші ознаки** та **уникнути дублювання змінних**, що важливо для побудови ефективної моделі.


In [ ]:
sns.set(rc={'figure.figsize':(5,5)})
plot_co = sns.scatterplot(data=df.loc[:, ['CO(GT)', 'C6H6(GT)']],
                          x='CO(GT)',
                          y='C6H6(GT)')
plot_co.set_title('The relationship between CO(GT) x, C6H6(GT) y')

In [ ]:
colname="NO2(GT)"
col_sd = df[colname].std()
col_mean = df[colname].mean()
x = (df[colname] - col_mean)/col_sd
print(col_sd)
sns.displot(x=x, kde=True, stat='density', )

In [ ]:
df[x > 4]

In [ ]:
quantile = df[colname].quantile([0.75])
df.loc[x > 4, [colname]] = quantile.iloc[0]

In [ ]:
df.head()

# Feature Engineering

Цікавий факт: PT08.S2(NMHC) має дуже сильний квадратичний або

---

експоненціальний звязок з C6H6(GT). <br>

In [ ]:
sns.scatterplot(x = df['PT08.S2(NMHC)'], y = df['C6H6(GT)'])

In [ ]:
sns.scatterplot(x = df['PT08.S2(NMHC)']**2, y = df['C6H6(GT)'])

Давайте перевіримо кореляцію Пірсона і її p-values для квадрату PT08.S2(NMHC) і C6H6(GT)

In [ ]:
st.pearsonr(df['PT08.S2(NMHC)']**2, df['C6H6(GT)'])

Як бачимо, маємо майже лінійну залежність і p-value = 0, що означає, що цей звязок є значущим і лінійним. <br>
Згідно з вигомами відсутності мультиколінеарності ми не можемо використовувати квадрат PT08.S2(NMHC)<br>
В іншому випадку нам достатньо лише цього параметру, щоб прогнозувати C6H6(GT) чи наш y.

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr

# Вибираємо тільки числові колонки
numeric_cols = df.select_dtypes(include=np.number).columns

# Порожній список для збереження результатів
results = []

# Перебираємо всі пари X і Y, де X ≠ Y
for col_x in numeric_cols:
    for col_y in numeric_cols:
        if col_x != col_y:
            x_squared = df[col_x] ** 2
            y = df[col_y]

            # Вилучаємо NaN
            mask = (~x_squared.isna()) & (~y.isna())
            if mask.sum() > 2:
                r, p = pearsonr(x_squared[mask], y[mask])
                results.append({
                    'X': col_x,
                    'Y': col_y,
                    'Corr(X², Y)': r,
                    'p-value': p
                })

# Перетворюємо в датафрейм
results_df = pd.DataFrame(results)


# Вивід результату
print(results_df.sort_values(by='Corr(X², Y)', ascending=False).head(10))


Додамо день тижня

In [ ]:
results_df.sort_values(by='Corr(X², Y)', ascending=False).head(20)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy.stats import pearsonr

top3 = [
    ('PT08.S2(NMHC)', 'C6H6(GT)'),
    ('PT08.S1(CO)', 'C6H6(GT)'),
    ('PT08.S1(CO)', 'PT08.S5(O3)')
]

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, (x_col, y_col) in enumerate(top3):
    x = df[x_col] ** 2
    y = df[y_col]
    mask = (~x.isna()) & (~y.isna())
    axes[i].scatter(x[mask], y[mask], alpha=0.5)
    axes[i].set_xlabel(f'{x_col}²')
    axes[i].set_ylabel(y_col)
    axes[i].set_title(f'{x_col}² vs {y_col}')

plt.tight_layout()
plt.show()

# Тестуємо лінійну модель: PT08.S2(NMHC)² → C6H6(GT)
x = df['PT08.S2(NMHC)'] ** 2
y = df['C6H6(GT)']
mask = (~x.isna()) & (~y.isna())
x_reshaped = x[mask].values.reshape(-1, 1)
y_values = y[mask].values

model = LinearRegression()
model.fit(x_reshaped, y_values)

# Результати моделі
r2 = model.score(x_reshaped, y_values)
coef = model.coef_[0]
intercept = model.intercept_

print(f"R² = {r2:.4f}")
print(f"Коефіцієнт нахилу (slope): {coef:.4f}")
print(f"Вільний член (intercept): {intercept:.4f}")


PT08.S2(NMHC)²має майже ідеальний лінійний зв'язок з C6H6(GT)

Це свідчить про дуже сильну залежність, яка може бути використана в якості єдиного предиктора в моделі.

PT08.S1(CO)² також корелює з C6H6(GT)

Зв'язок сильний, але менш чіткий, з більшим розсіюванням.

PT08.S1(CO)² має сильний зв’язок з PT08.S5(O3)

Можна використати для побудови моделі, проте можливі нелінійності.

Висновок:

Квадрати деяких змінних дозволяють виявити приховані лінійні залежності, що покращує вибір ознак для майбутнього моделювання.

In [ ]:
df['Weekday'] = df['Date'].dt.dayofweek

In [ ]:
colname = "Weekday"
print(st.pearsonr(df[colname], df['C6H6(GT)']))
sns.boxplot(x = df[colname], y = df['C6H6(GT)'])

In [ ]:
print(st.pearsonr(df[colname], df['C6H6(GT)']))


Значення -0.123 означає слабку, але статистично значущу від’ємну кореляцію.

Тобто: чим ближче до кінця тижня (субота-неділя), тим нижча середня концентрація бензолу.

Додамо револьверний індекс дня тижня

In [ ]:
df['Weekday_revo'] = abs(3 - df['Date'].dt.dayofweek)

Цей підхід перетворює тиждень у симетричну шкалу навколо середи:

Середа → abs(3 - 3) = 0

Вівторок і четвер → abs(3 - 2) = 1, abs(3 - 4) = 1

Понеділок і п’ятниця → 2

Субота і неділя → 3

Якщо припустити, що забруднення найбільше в середині тижня, ми можемо перевірити цю гіпотезу.

Потім можна побудувати графік або виконати регресію, щоб побачити, чи справді концентрація, наприклад, C6H6(GT) зменшується в міру віддалення від середи.

Додамо місяць

In [ ]:
df['Month'] = df['Date'].dt.month

In [ ]:
# Створення колонки "Season"
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Autumn'

df['Season'] = df['Month'].apply(get_season)


In [ ]:
df.columns

In [ ]:
df['sin_hour'] = np.sin(2 * np.pi * df['Hour']/24)
df['cos_hour'] = np.cos(2 * np.pi * df['Hour']/24)


In [ ]:
df['dew_point'] = df['T'] - ((100 - df['RH']) / 5)


In [ ]:
df['C6H6_roll3'] = df['C6H6(GT)'].rolling(window=3).mean()


In [ ]:
df['C6H6_lag1'] = df['C6H6(GT)'].shift(1)

In [ ]:
df.columns

In [ ]:
df.isnull().sum().sort_values(ascending=False)

In [ ]:
df = df.dropna(subset=['C6H6_roll3', 'C6H6_lag1'])

In [ ]:
df.isnull().sum().sort_values(ascending=False)

# Побудуємо моделі

In [ ]:
Y_df = df["C6H6(GT)"]
#  Skip "PT08.S2(NMHC)" as it has functional dependency on with the dependent variable "C6H6(GT)"
X_df = df.drop(labels=['C6H6(GT)', 'PT08.S2(NMHC)', 'Date', 'Time', 'DateTime'], inplace=False, axis=1)
X_df_c = X_df.copy()


In [ ]:
# Кодування колонки 'Season'
X_df_encoded = pd.get_dummies(X_df, columns=['Season'], drop_first=True)

In [ ]:
# split the data
X_train, X_test, y_train, y_test = train_test_split(X_df_encoded, Y_df, random_state=42)

In [ ]:
# define model
preprocessor = make_column_transformer(

    (Normalizer(), []), # do nothing with the data trasformation
    remainder="passthrough",
    verbose_feature_names_out=False,  # avoid to prepend the preprocessor names
)

model = make_pipeline(
    preprocessor,
    TransformedTargetRegressor(
        regressor=LinearRegression() # simple linear regression model, do nothing with the dependent variable
    ),
)

In [ ]:
# Кодування колонки 'Season'
X_df_encoded = pd.get_dummies(X_df, columns=['Season'], drop_first=True)

# Тепер заміни X_df на X_df_encoded
scores = cross_val_score(model, X_df_encoded, Y_df, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

# Отримаємо абсолютне MAE
scores = np.abs(scores)
print(f"Середній MAE: {scores.mean():.3f} (+/- {scores.std():.3f})")

In [ ]:
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, X_df_encoded, Y_df, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force scores to be positive
scores = np.absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

 Результат моделі:
Середнє абсолютне відхилення (MAE) склало 1.281 (±0.032)

Це означає, що в середньому модель помиляється на 1.28 одиниці при прогнозуванні концентрації бензолу C6H6(GT).

In [ ]:
#To improve mode performance, we can use a normalization and apply log to the target variable to turn it approximately into a normal distribution
# define model
preprocessor = make_column_transformer(
    (Normalizer(), X_df_encoded.columns), # normalize all features
    remainder="passthrough",
    verbose_feature_names_out=False,  # avoid to prepend the preprocessor names
)

model = make_pipeline(
    preprocessor,
    TransformedTargetRegressor(
        regressor=LinearRegression(), func=np.log10, inverse_func=sp.special.exp10 # apply log to target variable to turn it approximately into a normal distribution
    ),
)

In [ ]:
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, X_df_encoded, Y_df, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force scores to be positive
scores = np.absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

Трансформація np.log10(y) покращила модель, якщо раніше MAE було, наприклад, 1.28 (як у попередньому запуску).

тепер середня абсолютна помилка (MAE) ≈ 1.013


↑ We got a little improvement in the model performance. I don't what to do more for improving the model performance.

In [ ]:
# define model
preprocessor = make_column_transformer(

    (Normalizer(), []), # do nothing with the data trasformation
    remainder="passthrough",
    verbose_feature_names_out=False,  # avoid to prepend the preprocessor names
)

model_lasso = make_pipeline(
    preprocessor,
    TransformedTargetRegressor(
        regressor=Lasso(alpha=1.0) # simple linear regression model, do nothing with the dependent variable
    ),
)

In [ ]:
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model_lasso, X_df_encoded, Y_df, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force scores to be positive
scores = np.absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

Models such as ridge or lasso work best for a normal distribution of error.

In [ ]:
alphas = np.logspace(-10, 10, 21)

preprocessor = make_column_transformer(
    (Normalizer(), X_df_encoded.columns), #
    remainder="passthrough",
    verbose_feature_names_out=False,  # avoid to prepend the preprocessor names
)

model = make_pipeline(
    preprocessor,
    TransformedTargetRegressor(
        regressor=LassoCV(alphas=alphas),
        func=np.log10,
       inverse_func=sp.exp10),
)

model.fit(X_train, y_train)
model[-1].regressor_.alpha_

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import train_test_split
import numpy as np
import scipy.special as sp

# Створюємо діапазон значень alpha
alphas = np.logspace(-10, 10, 21)

# Масштабування ознак
preprocessor = make_column_transformer(
    (StandardScaler(), X_df_encoded.columns),
    remainder="passthrough",
    verbose_feature_names_out=False
)

# Побудова моделі
model = make_pipeline(
    preprocessor,
    TransformedTargetRegressor(
        regressor=LassoCV(alphas=alphas, max_iter=20_000),
        func=np.log10,
        inverse_func=sp.exp10
    )
)

# Навчання
model.fit(X_train, y_train)

# Вивід найкращого alpha
print("Найкраще alpha:", model[-1].regressor_.alpha_)


Найкраще alpha = 1e-10 означає, що модель фактично не потребує жорсткої регуляризації — вона вже добре підлаштовується під дані.
Проте варто слідкувати за переобученням або спробувати менший діапазон alphas.

LassoCV applies cross validation in order to determine which value of the regularization parameter (alpha) is best suited for prediction.
In our case, the best alpha is 1e-08. if we use model as Lasso(alpha=1e-08) we should get better results than using Lasso(alpha=1.0)

In [ ]:
# define model
preprocessor = make_column_transformer(

    (Normalizer(), []), # do nothing with the data trasformation
    remainder="passthrough",
    verbose_feature_names_out=False,  # avoid to prepend the preprocessor names
)

model_lasso = make_pipeline(
    preprocessor,
    TransformedTargetRegressor(
        regressor=Lasso(alpha=9.999999999999999e-11) # simple linear regression model, do nothing with the dependent variable
    ),
)

In [ ]:
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model_lasso, X_df_encoded, Y_df, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force scores to be positive
scores = np.absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

In [ ]:
#Evetually the model requires data transformation.
alpha = 9.999999999999999e-11

preprocessor = make_column_transformer(
    (Normalizer(), X_df_encoded.columns), #
    remainder="passthrough",
    verbose_feature_names_out=False,  # avoid to prepend the preprocessor names
)

model_lasso = make_pipeline(
    preprocessor,
    TransformedTargetRegressor(
        regressor=Lasso(alpha=alpha), func=np.log10, inverse_func=sp.exp10

    ),
)

# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model_lasso, X_df_encoded, Y_df, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force scores to be positive
scores = np.absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

Модель Lasso із логарифмічною трансформацією цільової змінної (log10) та найкращим значенням alpha ≈ 1e-10, знайденим через LassoCV, показала стабільний результат:

Середня похибка (MAE): 1.013

Стандартне відхилення: 0.033

Використано 30-кратну крос-валідацію (10 фолдів * 3 повторення)


Трансформація цільової змінної дозволила зробити розподіл більш наближеним до нормального.


Маленьке значення alpha означає, що регуляризація майже не "штрафує" коефіцієнти, отже модель не відкидає ознаки, але трохи згладжує вплив шуму.

Ridge model

In [ ]:
alphas = np.logspace(-10, 10, 21)

preprocessor = make_column_transformer(
    (Normalizer(), X_df_encoded.columns), #
    remainder="passthrough",
    verbose_feature_names_out=False,  # avoid to prepend the preprocessor names
)

model = make_pipeline(
    preprocessor,
    TransformedTargetRegressor(
        regressor=RidgeCV(alphas=alphas), func=np.log10, inverse_func=sp.exp10

    ),
)

model.fit(X_train, y_train)
model[-1].regressor_.alpha_

Модель RidgeCV із alpha=1e-07 показує, що дані добре піддаються моделюванню, і лише мінімальна регуляризація допомагає уникнути перенавчання. Така мала регуляризація свідчить про низьку мультиколінеарність після обробки та масштабування ознак.


In [ ]:
#Evetually the model requires data transformation.
alpha = 1e-08

preprocessor = make_column_transformer(
    (Normalizer(), X_df_encoded.columns), #
    remainder="passthrough",
    verbose_feature_names_out=False,  # avoid to prepend the preprocessor names
)

model_ridge = make_pipeline(
    preprocessor,
    TransformedTargetRegressor(
        regressor=Ridge(alpha=alpha), func=np.log10, inverse_func=sp.exp10

    ),
)

# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model_ridge, X_df_encoded, Y_df, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force scores to be positive
scores = np.absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

Модель з alpha=1e-08 дає стабільний результат, дуже схожий на результат моделі Lasso.

Логарифмічна трансформація допомогла зробити розподіл цільової змінної більш нормальним, що покращило точність.

Низьке значення alpha свідчить про мінімальну потребу в регуляризації, тобто твоя модель не переобтяжена складністю і дані добре підготовлені.


In [ ]:
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, X_df_encoded, Y_df, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force scores to be positive
scores = np.absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

We can check the variability of the coefficients across folds to make sure that there isn't a correlation between the features.

In [ ]:
cv = RepeatedKFold(n_splits=8, n_repeats=10, random_state=0)
model_ridge.fit(X_train, y_train)
cv_model = cross_validate(
    model_ridge,
    X_df_encoded,
    Y_df,
    cv=cv,
    return_estimator=True,
    scoring='neg_mean_absolute_error',
    n_jobs=-1
)

coefs = pd.DataFrame(
    [
        est[-1].regressor_.coef_ * est[:-1].transform(X_df_encoded.iloc[train_idx]).std(axis=0)
        for est, (train_idx, _) in zip(cv_model["estimator"], cv.split(X_df, Y_df))
    ],
    columns=model_ridge[:-1].get_feature_names_out(), # feature names
)

plt.figure(figsize=(9, 7))
sns.stripplot(data=coefs, orient="h", palette="dark:k", alpha=0.5)
sns.boxplot(data=coefs, orient="h", color="cyan", saturation=0.5, whis=10)
plt.axvline(x=0, color=".5")
plt.xlabel("Coefficient importance")
plt.title("Coefficient importance and its variability")
plt.suptitle("Ridge model, small regularization")
plt.subplots_adjust(left=0.3)

Coefficients do not have a big variance, so they are more or less stable.

---

# Загальний висновок про важливість ознак (Ridge-регресія)

На основі аналізу коефіцієнтів моделі Ridge з малою регуляризацією:

##  Найінформативніші ознаки:
Ці ознаки мають стабільні коефіцієнти, що знаходяться далеко від нуля:

- `T` — температура: **найсильніший негативний вплив**  
- `RH` — відносна вологість: **негативний вплив**  
- `AH` — абсолютна вологість: **значний вплив**  
- `PT08.S4(NO2)` і `PT08.S5(O3)` — **найсильніші позитивні предиктори**  
- `C6H6_lag1` — попереднє значення бензолу: **сильний позитивний вплив**  
- `dew_point` — точка роси: також значуща ознака  

## Слабші або нестабільні ознаки:
Мають коефіцієнти, близькі до нуля, або сильно варіативні:

- `sin_hour`, `cos_hour` — **добова циклічність не має значного впливу**
- `Weekday`, `Month` — **часові ознаки мають мінімальний ефект**
- `Season_*` — сезонність має **помірний стабільний вплив**

## Рекомендації:
- **Зберегти** тільки інформативні та стабільні ознаки для побудови простої та надійної моделі.
- **Розглянути зменшення розмірності** або **регуляризацію**, щоб зменшити шум від слабких фіч.
- **Добова циклічність** може бути виключена без втрати якості.

---

Polinomial model

In [ ]:
# polinomial model
preprocessor = make_column_transformer(
    (PolynomialFeatures(2), X_df_encoded.columns), #
    remainder="passthrough",
    verbose_feature_names_out=False,  # avoid to prepend the preprocessor names
)

model_plnm = make_pipeline(
    preprocessor,
    TransformedTargetRegressor(
        regressor=LinearRegression()

    ),
)

# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model_plnm, X_df_encoded, Y_df, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force scores to be positive
scores = np.absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

In [ ]:
from sklearn.model_selection import cross_validate, RepeatedKFold
import numpy as np

# 1. Побудова моделі
poly = PolynomialFeatures(2, include_bias=False)
preprocessor = make_column_transformer(
    (poly, X_df_encoded.columns),
    remainder="passthrough",
    verbose_feature_names_out=False
)

model_plnm = make_pipeline(
    preprocessor,
    TransformedTargetRegressor(
        regressor=LinearRegression()
    )
)

# 2. Крос-валідація
cv = RepeatedKFold(n_splits=8, n_repeats=10, random_state=0)
cv_model = cross_validate(
    model_plnm,
    X_df_encoded,
    Y_df,
    cv=cv,
    return_estimator=True,
    scoring='neg_mean_absolute_error',
    n_jobs=-1
)

# 3. Отримуємо feature names з ПЕРШОЇ навченої моделі
feature_names = cv_model['estimator'][0].named_steps['columntransformer'].get_feature_names_out()

# 4. Коефіцієнти для кожного спліта
coef_matrix = pd.DataFrame(
    [est.named_steps['transformedtargetregressor'].regressor_.coef_ for est in cv_model['estimator']],
    columns=feature_names
)

# 5. Візуалізація
mean_importance = coef_matrix.mean(axis=0).abs().sort_values(ascending=False)

# Обираємо топ 30 найважливіших
top_features = mean_importance.head(30).index

# Фільтруємо тільки ці ознаки
coef_filtered = coef_matrix[top_features]

# Малюємо графік
plt.figure(figsize=(10, 8))
sns.stripplot(data=coef_filtered, orient="h", color='black', alpha=0.4)
sns.boxplot(data=coef_filtered, orient="h", color="lime", saturation=0.5, whis=10)
plt.axvline(x=0, color="gray", linestyle='--')
plt.xlabel("Важливість коефіцієнта")
plt.title("🔝 Топ-30 важливих поліноміальних ознак")
plt.subplots_adjust(left=0.35)
plt.show()



---

###  **Висновки за результатами моделювання**

У ході виконання завдання було проведено повний цикл аналізу даних якості повітря, включно з обробкою пропущених значень, інженерією ознак та побудовою різних моделей регресії для передбачення концентрації бензолу `C6H6(GT)`.

###  1. **Базові моделі**

- **Лінійна регресія (без трансформацій)**:  
  `MAE ≈ 1.281`, модель демонструє базову точність, однак не враховує особливості розподілу цільової змінної.

- **З логарифмічною трансформацією цільової змінної (log10)**:  
  `MAE ≈ 1.013`, покращення точності за рахунок нормалізації розподілу.

---

###  2. **Регуляризовані моделі**

- **LassoCV** (автоматичний підбір alpha):  
  Найкраще `alpha = 1e-10`, що означає **мінімальну регуляризацію**.  
  З логарифмічною трансформацією: `MAE ≈ 1.013`  
  Без трансформації: `MAE ≈ 1.281`

- **RidgeCV**:  
  Подібна до Lasso за точністю (`MAE ≈ 1.013`), стабільні коефіцієнти без перенавчання.

 *Висновок*: Обидві моделі показали **стабільну якість**, однак не змогли суттєво зменшити похибку нижче 1.0.

---

###  3. **Поліноміальна модель (PolynomialFeatures(2))**

- Побудована модель другого ступеня з усіма взаємодіями ознак.
- Показала **найкращий результат серед усіх моделей**:  
   `Mean MAE ≈ 0.687 (±0.028)`  
- Це підтверджує, що **нелінійні зв’язки між змінними критично важливі** для точного прогнозування `C6H6(GT)`.

---

###  4. **Важливість ознак**

- За допомогою **Ridge-регресії** та аналізу коефіцієнтів було виявлено:
  - Найбільш значущі ознаки: `T`, `RH`, `AH`, `PT08.S4(NO2)`, `PT08.S5(O3)`, `C6H6_lag1`, `dew_point`
  - Незначущі: `sin_hour`, `cos_hour`, `Weekday`, `Month` — їх можна виключити

- В моделі з **поліноміальними ознаками**:
  - Найважливішими стали: `AH²`, `AH * Season_Winter`, `cos_hour²`, `sin_hour * cos_hour`, `C6H6_lag1 * AH`

---

###  **Загальний підсумок**

| Модель                  | MAE       | Коментар |
|------------------------|-----------|----------|
| Лінійна регресія       | 1.281     | Базовий рівень |
| З логарифмом (log10)   | 1.013     | Покращення за рахунок трансформації |
| Lasso (α ≈ 1e-10)       | 1.013     | Автоматичний відбір ознак |
| Ridge (α ≈ 1e-08)       | 1.013     | Стабільна модель з малою регуляризацією |
| **Поліноміальна модель** | **0.687** | `Найкраща модель, враховує взаємодії та квадратичні зв’язки` |

---

###  **Рекомендації**:

- Використовувати **поліноміальну модель** другого ступеня — вона забезпечує найменшу похибку.
- Для побудови інтерпретованих моделей — обирати **Ridge або Lasso** з логарифмічною трансформацією.
- Можна **зменшити кількість ознак** на основі аналізу важливості — це спростить модель без втрати якості.

---

###Вплив зменшення розмірності на якість моделі

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.compose import make_column_transformer
from sklearn.model_selection import RepeatedKFold, cross_val_score
from sklearn.compose import TransformedTargetRegressor
import numpy as np

# Побудова повного пайплайну з SelectFromModel всередині
poly = PolynomialFeatures(degree=2, include_bias=False)

preprocessor = make_column_transformer(
    (poly, X_df_encoded.columns),
    remainder="passthrough",
    verbose_feature_names_out=False
)

selector = SelectFromModel(LinearRegression(), threshold=0.001)

model_reduced_pipeline = make_pipeline(
    preprocessor,
    selector,
    TransformedTargetRegressor(regressor=LinearRegression())
)

# Оцінювання
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(model_reduced_pipeline, X_df_encoded, Y_df,
                         scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

scores = np.abs(scores)
print(f" Mean MAE (спрощена модель): {scores.mean():.3f} ± {scores.std():.3f}")


Використання SelectFromModel з порогом 0.001 дозволило:

значно зменшити кількість ознак

майже не погіршити якість моделі

знизити ризик перенавчання (overfitting)

спростити інтерпретацію і підвищити стабільність

Поріг threshold=0.001 —  відкидає слабкі фічі, але зберігає важливі взаємодії

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.compose import make_column_transformer
from sklearn.compose import TransformedTargetRegressor
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Побудова трансформера
poly = PolynomialFeatures(degree=2, include_bias=False)
preprocessor = make_column_transformer(
    (poly, X_df_encoded.columns),
    remainder="passthrough",
    verbose_feature_names_out=False
)

# Створення окремого пайплайну без cross_val для витягу фіч
model_for_visualization = make_pipeline(
    preprocessor,
    SelectFromModel(LinearRegression(), threshold=0.1),
    TransformedTargetRegressor(regressor=LinearRegression())
)

# Навчання моделі
model_for_visualization.fit(X_df_encoded, Y_df)

# Отримання фіч після трансформації
feature_names = model_for_visualization.named_steps['columntransformer'].get_feature_names_out()

# Отримання маски вибраних ознак
mask = model_for_visualization.named_steps['selectfrommodel'].get_support()

# Отримання відібраних фіч і їх коефіцієнтів
selected_features = feature_names[mask]
selected_coefs = model_for_visualization.named_steps['transformedtargetregressor'].regressor_.coef_

# Побудова датафрейму
coef_df = pd.DataFrame({
    'Feature': selected_features,
    'Importance': selected_coefs
}).sort_values(by='Importance', key=abs, ascending=False)

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(model_for_visualization, X_df_encoded, Y_df,
                         scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

scores = np.abs(scores)
print(f" Mean MAE (спрощена модель): {scores.mean():.3f} ± {scores.std():.3f}")


plt.figure(figsize=(10, 8))
sns.barplot(data=coef_df, y='Feature', x='Importance', palette='viridis')
plt.title('Найважливіші відібрані ознаки (SelectFromModel + PolyFeatures)')
plt.tight_layout()
plt.show()


In [ ]:
# Визначаємо ознаки, які хочемо виключити
cols_to_drop = ['Hour', 'sin_hour', 'cos_hour', 'CO(GT)', 'PT08.S1(CO)']

# Видаляємо їх з ознак
X_df_reduced = X_df_encoded.drop(columns=cols_to_drop)


In [ ]:

poly = PolynomialFeatures(degree=2, include_bias=False)
preprocessor = make_column_transformer(
    (poly, X_df_reduced.columns),
    remainder="passthrough",
    verbose_feature_names_out=False
)

selector = SelectFromModel(LinearRegression(), threshold=0.00001)

model_final_pipeline = make_pipeline(
    preprocessor,
    selector,
    TransformedTargetRegressor(regressor=LinearRegression())
)

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(model_final_pipeline, X_df_reduced, Y_df,
                         scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

print(f"Mean MAE (без CO та часових ознак): {np.mean(np.abs(scores)):.3f} ± {np.std(scores):.3f}")


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.compose import make_column_transformer
from sklearn.compose import TransformedTargetRegressor

# Побудова моделі з найкращими параметрами
poly = PolynomialFeatures(degree=2, include_bias=False)

preprocessor = make_column_transformer(
    (poly, X_df_reduced.columns),
    remainder="passthrough",
    verbose_feature_names_out=False
)

selector = SelectFromModel(LinearRegression(), threshold=0.00001)

model_final_pipeline = make_pipeline(
    preprocessor,
    selector,
    TransformedTargetRegressor(regressor=LinearRegression())
)

# Навчання моделі на всіх даних
model_final_pipeline.fit(X_df_reduced, Y_df)

# Отримання імен ознак після трансформації
feature_names = model_final_pipeline.named_steps['columntransformer'].get_feature_names_out()

# Отримання маски вибраних ознак
selected_mask = model_final_pipeline.named_steps['selectfrommodel'].get_support()

# Отримання коефіцієнтів тільки для вибраних ознак
selected_features = feature_names[selected_mask]
coefs = model_final_pipeline.named_steps['transformedtargetregressor'].regressor_.coef_

# Побудова датафрейму
coef_df = pd.DataFrame({
    'Feature': selected_features,
    'Importance': coefs
}).sort_values(by='Importance', key=np.abs, ascending=False)

# Візуалізація
plt.figure(figsize=(10, 10))
sns.barplot(data=coef_df, y='Feature', x='Importance', palette='viridis')
plt.title('🔍 Найважливіші залишені ознаки (SelectFromModel + PolyFeatures)')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.tight_layout()
plt.show()


In [ ]:
X_df_reduced.columns

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.compose import make_column_transformer
from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import RepeatedKFold, cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

selected_columns = [
    'NOx(GT)', 'PT08.S3(NOx)', 'NO2(GT)',
    'PT08.S4(NO2)', 'PT08.S5(O3)', 'T', 'AH',
    'Weekday', 'C6H6_roll3', 'C6H6_lag1', 'Weekday_revo'
]

# Припустимо, X_df та Y_df вже визначено в сесії (оскільки вони раніше використовувались)
X_df_final = X_df[selected_columns]

# Побудова пайплайну
poly = PolynomialFeatures(degree=2, include_bias=False)
preprocessor = make_column_transformer(
    (poly, X_df_final.columns),
    remainder="passthrough",
    verbose_feature_names_out=False
)

selector = SelectFromModel(LinearRegression(), threshold=0.00001)

model_final_pipeline = make_pipeline(
    preprocessor,
    selector,
    TransformedTargetRegressor(regressor=LinearRegression())
)

# Крос-валідація
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(model_final_pipeline, X_df_final, Y_df,
                         scoring='neg_mean_absolute_error',
                         cv=cv, n_jobs=-1)



coef_df = pd.DataFrame({
    'Feature': np.array(feature_names)[mask],
    'Importance': coefs[mask]
}).sort_values(by='Importance', key=abs, ascending=False)

# Побудова графіку
plt.figure(figsize=(10, 8))
sns.barplot(data=coef_df, y='Feature', x='Importance', palette='viridis')
plt.title('💡 Найважливіші залишені ознаки (SelectFromModel + PolyFeatures)')
plt.tight_layout()
plt.show()

# Виведення MAE
mean_mae = np.mean(np.abs(scores))
std_mae = np.std(scores)
mean_mae, std_mae
print(f"Mean MAE (без CO та часових ознак): {np.mean(np.abs(scores)):.3f} ± {np.std(scores):.3f}")



### **Висновок щодо ролі абсолютної вологості (AH) у прогнозуванні концентрації бензолу (C6H6):**

Абсолютна вологість (AH) є однією з найважливіших ознак у моделі прогнозування, і ця висока важливість має як статистичне, так і фізико-хімічне обґрунтування:

###  1. **Наукове підґрунтя:**
Ось деякі наукові дослідження, які вивчають вплив вологості на концентрацію бензолу в атмосфері:

1. **"The competing role of moisture in adsorption of gaseous benzene on microporous activated carbon"**  
   *Chemical Engineering Journal*.  
   Дослідження показує, що волога негативно впливає на адсорбцію бензолу на активованому вугіллі, особливо при низьких концентраціях бензолу.  
   [Посилання](https://www.sciencedirect.com/science/article/abs/pii/S1383586621011953)

2. **"The dynamic competition in adsorption between gaseous benzene and water vapor on hydrophilic MOF-199"**  
   *Journal of Hazardous Materials*.  
   Виявлено, що волога має негативний вплив на адсорбцію бензолу, особливо на MOF-199 при низьких рівнях бензолу.  
   [Посилання](https://www.sciencedirect.com/science/article/abs/pii/S1385894720339334)

3. **"Influence of temperature and humidity on the detection of benzene vapor by piezoelectric crystal sensor"**  
   *arXiv*.  
   Дослідження вивчає вплив температури та вологості на виявлення парів бензолу за допомогою п'єзоелектричного кристалічного сенсора.  
   [Посилання](https://arxiv.org/abs/1611.05958)

Ці дослідження підтверджують, що підвищена вологість може знижувати ефективність видалення бензолу з атмосфери, оскільки молекули води конкурують з молекулами бензолу за активні місця на поверхнях адсорбентів.


###  2. **Модельне підтвердження:**
- AH має **найвищу або одну з найвищих важливостей** у моделі з `PolynomialFeatures + SelectFromModel`, що означає сильний зв’язок між вологістю та концентрацією цільового забруднювача.
- Висока важливість AH спостерігається стабільно незалежно від того, які інші змінні залишаються в моделі, що свідчить про її **незалежну прогностичну силу**.



### ✅ 3. **Практичне значення:**
- AH — не лише погодна змінна, але й **індикатор умов, що впливають на хімічні реакції та утримання газів** у повітрі.
- Її включення в модель покращує точність, зменшуючи похибку (MAE)ю.



Таким чином, абсолютна вологість — це не просто "фоновий" метеопараметр, а **ключовий фізико-хімічний фактор**, що впливає на атмосферну поведінку бензолу. Її важливість обґрунтована як емпіричними результатами моделі, так і дослідженнями з літератури.


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.compose import make_column_transformer, TransformedTargetRegressor
from sklearn.model_selection import RepeatedKFold, cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# Використовуємо обрані ознаки
selected_columns = [
    'NOx(GT)', 'PT08.S3(NOx)', 'NO2(GT)',
    'PT08.S4(NO2)', 'PT08.S5(O3)',
    'Weekday', 'Weekday_revo',
    'C6H6_roll3', 'C6H6_lag1'
]
X_df_final = X_df[selected_columns]

# Побудова моделі
poly = PolynomialFeatures(degree=2, include_bias=False)
preprocessor = make_column_transformer(
    (poly, X_df_final.columns),
    remainder="passthrough",
    verbose_feature_names_out=False
)

selector = SelectFromModel(LinearRegression(), threshold=0.00001)

model_pipeline = make_pipeline(
    preprocessor,
    selector,
    TransformedTargetRegressor(regressor=LinearRegression())
)

# Крос-валідація
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(model_pipeline, X_df_final, Y_df,
                         scoring='neg_mean_absolute_error',
                         cv=cv, n_jobs=-1)

# Навчання моделі для отримання важливостей
model_pipeline.fit(X_df_final, Y_df)

# Витяг ознак та коефіцієнтів
feature_names = model_pipeline.named_steps['columntransformer'].get_feature_names_out()
coefs = model_pipeline.named_steps['selectfrommodel'].estimator_.coef_
mask = model_pipeline.named_steps['selectfrommodel'].get_support()

# Побудова таблиці з важливостями
coef_df = pd.DataFrame({
    'Feature': np.array(feature_names)[mask],
    'Importance': coefs[mask]
}).sort_values(by='Importance', key=abs, ascending=False).head(10)

# Побудова графіка
plt.figure(figsize=(10, 7))
sns.barplot(data=coef_df, y='Feature', x='Importance', palette='viridis')
plt.title('Топ-10 найважливіших ознак (без T, без AH, без dew_point)')
plt.tight_layout()
plt.grid(True, axis='x')
plt.show()

# Повертаємо MAE
np.mean(np.abs(scores)), np.std(scores)


In [ ]:
import shap
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import PartialDependenceDisplay

# Обираємо тільки dew_point та AH для демонстрації впливу
selected_columns_for_shap = ['dew_point', 'AH']
X_for_shap = X_df[selected_columns_for_shap]

# Побудова моделі для SHAP (використаємо RandomForest для кращого представлення нелінійностей)
rf_model = Pipeline([
    ("scaler", StandardScaler()),
    ("regressor", RandomForestRegressor(n_estimators=100, random_state=42))
])

# Навчання моделі
rf_model.fit(X_for_shap, Y_df)

# SHAP values
explainer = shap.Explainer(rf_model.named_steps['regressor'], X_for_shap)
shap_values = explainer(X_for_shap)

# Partial Dependence Plot
fig_pdp, ax = plt.subplots(figsize=(10, 4))
PartialDependenceDisplay.from_estimator(
    rf_model, X_for_shap, features=[0, 1], ax=ax
)

# SHAP Summary Plot (only for dew_point and AH)
shap.summary_plot(shap_values.values, X_for_shap, feature_names=selected_columns_for_shap, show=False)
plt.tight_layout()

plt.show()
fig_pdp.show()


In [ ]:
from sklearn.inspection import PartialDependenceDisplay
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.compose import make_column_transformer, TransformedTargetRegressor
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Використаємо останню модель з dew_point та AH для PDP
selected_columns = [
    'NOx(GT)', 'PT08.S3(NOx)', 'NO2(GT)',
    'PT08.S4(NO2)', 'PT08.S5(O3)', 'T', 'AH',
    'Weekday', 'C6H6_roll3', 'C6H6_lag1', 'Weekday_revo', 'dew_point'
]

X_selected = X_df[selected_columns]
X_train, X_test, y_train, y_test = train_test_split(X_selected, Y_df, test_size=0.2, random_state=42)

# Побудова моделі
poly = PolynomialFeatures(degree=2, include_bias=False)
preprocessor = make_column_transformer(
    (poly, X_selected.columns),
    remainder="passthrough",
    verbose_feature_names_out=False
)

selector = SelectFromModel(LinearRegression(), threshold=0.00001)

pipeline = make_pipeline(
    preprocessor,
    selector,
    TransformedTargetRegressor(regressor=LinearRegression())
)

# Навчання моделі
pipeline.fit(X_train, y_train)

# Побудова PDP для AH та dew_point
features_for_pdp = ['AH', 'dew_point']
fig, ax = plt.subplots(figsize=(10, 5))
PartialDependenceDisplay.from_estimator(pipeline, X_train, features_for_pdp, ax=ax)
plt.tight_layout()
plt.show()


###Висновок щодо вибору ознаки вологості:

Модель показує, що точка роси (dew_point) є більш інформативною ознакою, ніж абсолютна вологість (AH). Це підтверджується:

SHAP-аналізом — dew_point має сильний і стабільний негативний вплив на концентрацію бензолу.

PDP-графіком — спостерігається чіткий спад тренду: при зростанні точки роси модель очікує зниження вмісту бензолу.

Фізичною інтерпретацією — точка роси краще відображає наявність вологи в атмосфері у зв'язку з температурою, і, отже, визначає умови для адсорбції та фотохімічних процесів.

Практичний висновок:

З огляду на стабільний вплив, наочну фізичну інтерпретацію та підтримку в літературі, доцільно залишити саме dew_point як представника вологісних умов у моделі, замість більш змінного й менш стабільного AH.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.compose import make_column_transformer, TransformedTargetRegressor
from sklearn.model_selection import RepeatedKFold, cross_val_score

protected = ['dew_point']

selected_columns = [
    'NOx(GT)', 'PT08.S3(NOx)', 'NO2(GT)',
    'PT08.S4(NO2)', 'PT08.S5(O3)',
    'dew_point', 'Weekday', 'Weekday_revo',
    'C6H6_roll3', 'C6H6_lag1'
]

corr_matrix = X_df[selected_columns].corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

to_drop = []
threshold = 0.9
for column in upper.columns:
    for row in upper.index:
        if upper.loc[row, column] > threshold:
            if column not in protected and row not in protected:
                to_drop.append(column)

to_drop = list(set(to_drop))
print(" Видаляємо мультиколінеарні ознаки (крім dew_point):", to_drop)

selected_columns_updated = [col for col in selected_columns if col not in to_drop]
X_df_final = X_df[selected_columns_updated]

poly = PolynomialFeatures(degree=2, include_bias=False)
preprocessor = make_column_transformer(
    (poly, X_df_final.columns),
    remainder="passthrough",
    verbose_feature_names_out=False
)

selector = SelectFromModel(LinearRegression(), threshold=0.00001)

model_pipeline = make_pipeline(
    preprocessor,
    selector,
    TransformedTargetRegressor(regressor=LinearRegression())
)

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(model_pipeline, X_df_final, Y_df,
                         scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

model_pipeline.fit(X_df_final, Y_df)

feature_names = model_pipeline.named_steps['columntransformer'].get_feature_names_out()
coefs = model_pipeline.named_steps['selectfrommodel'].estimator_.coef_
mask = model_pipeline.named_steps['selectfrommodel'].get_support()

coef_df = pd.DataFrame({
    'Feature': np.array(feature_names)[mask],
    'Importance': coefs[mask]
}).sort_values(by='Importance', key=abs, ascending=False).head(10)

plt.figure(figsize=(10, 7))
sns.barplot(data=coef_df, y='Feature', x='Importance', palette='viridis')
plt.title('Топ-10 важливих ознак після видалення мультиколінеарних (dew_point збережено)')
plt.tight_layout()
plt.grid(True, axis='x')
plt.show()

print(f"MAE після очищення (з dew_point): {np.mean(np.abs(scores)):.3f} ± {np.std(scores):.3f}")


Під час побудови регресійної моделі було проведено аналіз мультиколінеарності з порогом 0.9. Надмірно корельовані ознаки були видалені, окрім dew_point, який збережено через його фізичну обґрунтованість та значущість у моделі. Додано поліноміальні ознаки другого ступеня для врахування нелінійних залежностей. Для відбору важливих фіч використано SelectFromModel з лінійною регресією. Модель перевірено за допомогою повторної крос-валідації, отримано MAE = 1.132 ± 0.034.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.compose import make_column_transformer, TransformedTargetRegressor
from sklearn.model_selection import RepeatedKFold, cross_val_score


selected_columns = [
    'NOx(GT)', 'PT08.S3(NOx)', 'NO2(GT)',
    'PT08.S4(NO2)', 'PT08.S5(O3)',
    'RH', 'dew_point',
    'Weekday', 'Weekday_revo',
    'C6H6_roll3', 'C6H6_lag1'
]

corr_matrix = X_df[selected_columns].corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

to_drop = []
threshold = 0.95
for column in upper.columns:
    for row in upper.index:
        if upper.loc[row, column] > threshold:
          to_drop.append(column)

to_drop = list(set(to_drop))
print(" Видаляємо мультиколінеарні ознаки (крім dew_point):", to_drop)

selected_columns_updated = [col for col in selected_columns if col not in to_drop]
X_df_final = X_df[selected_columns_updated]

poly = PolynomialFeatures(degree=2, include_bias=False)
preprocessor = make_column_transformer(
    (poly, X_df_final.columns),
    remainder="passthrough",
    verbose_feature_names_out=False
)

selector = SelectFromModel(LinearRegression(), threshold=0.00001)

model_pipeline = make_pipeline(
    preprocessor,
    selector,
    TransformedTargetRegressor(regressor=LinearRegression())
)

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(model_pipeline, X_df_final, Y_df,
                         scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

model_pipeline.fit(X_df_final, Y_df)

feature_names = model_pipeline.named_steps['columntransformer'].get_feature_names_out()
coefs = model_pipeline.named_steps['selectfrommodel'].estimator_.coef_
mask = model_pipeline.named_steps['selectfrommodel'].get_support()

coef_df = pd.DataFrame({
    'Feature': np.array(feature_names)[mask],
    'Importance': coefs[mask]
}).sort_values(by='Importance', key=abs, ascending=False).head(10)

plt.figure(figsize=(10, 7))
sns.barplot(data=coef_df, y='Feature', x='Importance', palette='viridis')
plt.title('Топ-10 важливих ознак після видалення мультиколінеарних')
plt.tight_layout()
plt.grid(True, axis='x')
plt.show()

print(f"MAE після очищення (з  відносна вологість): {np.mean(np.abs(scores)):.3f} ± {np.std(scores):.3f}")


Під час побудови моделі було виявлено мультиколінеарні ознаки за допомогою кореляційної матриці з порогом 0.95. Надмірно корельовані ознаки видалялися, однак RH (відносна вологість) було усвідомлено збережено через її високу значущість у фізичному та статистичному сенсі.

Було застосовано поліноміальне розширення ознак (PolynomialFeatures) для врахування взаємодій та нелінійностей. Надалі, SelectFromModel на основі LinearRegression дозволив залишити лише найбільш інформативні фічі.

Модель перевірено за допомогою Repeated K-Fold Cross-Validation, що дозволило уникнути переобучення. Отримано MAE = 0.869 ± 0.030, що свідчить про високу точність та стабільність.

Ключовий момент — збереження RH та dew_point як важливих показників атмосферного стану, навіть якщо вони потенційно мультиколінеарні, оскільки вони покращують модель як аналітично, так і логічно.